In [22]:
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import os
from azure.ai.projects import AIProjectClient
from azure.core.credentials import AzureKeyCredential 
from azure.ai.agents.models import FunctionTool, ToolSet, ListSortOrder
from azure.ai.agents.models import AgentsNamedToolChoice, AgentsNamedToolChoiceType, FunctionName
from azure.search.documents.agent import KnowledgeAgentRetrievalClient
from azure.search.documents.agent.models import KnowledgeAgentRetrievalRequest, KnowledgeAgentMessage, KnowledgeAgentMessageTextContent, KnowledgeAgentIndexParams


In [23]:


load_dotenv(override=True) # take environment variables from .env.


project_endpoint = os.environ["PROJECT_ENDPOINT"]
agent_model = os.getenv("AGENT_MODEL", "gpt-4.1-mini")
endpoint = os.environ["AZURE_SEARCH_ENDPOINT"]
openai_key = os.getenv("AZURE_OPENAI_KEY")
managed_identity_client_id = os.getenv("MANAGED_IDENTITY_CLIENT_ID")
new_credential = AzureKeyCredential(openai_key)
credential = DefaultAzureCredential(managed_identity_client_id=managed_identity_client_id)
# token_provider = get_bearer_token_provider(credential, "https://search.azure.com/.default")
index_name = os.getenv("AZURE_SEARCH_INDEX", "txt_files_index")
azure_openai_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"]
azure_openai_gpt_deployment = os.getenv("AZURE_OPENAI_GPT_DEPLOYMENT", "gpt-4.1-mini")
azure_openai_gpt_model = os.getenv("AZURE_OPENAI_GPT_MODEL", "gpt-4.1-mini")
azure_openai_embedding_deployment = os.getenv("AZURE_OPENAI_EMBEDDING_DEPLOYMENT", "text-embedding-3-large")
azure_openai_embedding_model = os.getenv("AZURE_OPENAI_EMBEDDING_MODEL", "text-embedding-3-large")
agent_name = os.getenv("AZURE_SEARCH_AGENT_NAME", "txt-files-agent")

In [24]:

print(f"Agent Name: {agent_name}")

Agent Name: txt-files-agent


In [25]:
project_client = AIProjectClient(endpoint=project_endpoint, credential=credential)

list(project_client.agents.list_agents())

[{'id': 'asst_1BNPct7pGpMEM1XPcVo99uEw', 'object': 'assistant', 'created_at': 1754215309, 'name': 'earth-search-agent-storage', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperature': 1.0, 'tool_resources': {}, 'metadata': {}, 'response_format': 'auto'},
 {'id': 'asst_K4j85U9fQl9Z4i4tb2ZbJbfg', 'object': 'assistant', 'created_at': 1754215159, 'name': 'earth-search-agent-storage', 'description': None, 'model': 'gpt-4.1-mini', 'instructions': '\nA Q&A agent that can answer questions about the Earth at night.\nSources have a JSON format with a ref_id that must be cited in the answer using the format [ref_id].\nIf you do not have the answer, respond with "I don\'t know".\n', 'tools': [], 'top_p': 1.0, 'temperatu

In [26]:
instructions = """
A Q&A agent that can answer questions about Earth at night, including topics like:
- Nighttime satellite imagery and observations
- Urban lighting patterns and light pollution
- Nocturnal ecosystems and wildlife
- Disaster monitoring using night imagery
- Human activity patterns visible from space
- Climate monitoring and urban heat island effects
- Bioluminescence in marine environments

Sources are text documents that have been processed into chunks. Each source has an 'id' field that must be cited in your answer using the format [id].
When referencing information, always cite the source using the document ID.
If you do not have the answer in the provided sources, respond with "I don't know".

Be comprehensive in your answers and cite multiple relevant sources when available.
"""
agent = project_client.agents.create_agent(
    model=agent_model,
    name=agent_name,
    instructions=instructions
)

print(f"AI agent '{agent_name}' created or updated successfully")

AI agent 'txt-files-agent' created or updated successfully


In [27]:
agent_client = KnowledgeAgentRetrievalClient(endpoint=endpoint, agent_name=agent_name, credential=new_credential)

thread = project_client.agents.threads.create()
retrieval_results = {}

def agentic_retrieval() -> str:
    """
    Searches text documents about Earth at night topics including satellite imagery, urban lighting, 
    nocturnal ecosystems, disaster monitoring, and climate science.
    The returned data contains text chunks with unique document IDs.
    Be sure to cite sources using the document ID format [id] in your agent's response.
    """
    # Take the last 5 messages in the conversation
    messages = project_client.agents.messages.list(thread.id, limit=5, order=ListSortOrder.DESCENDING)
    # Reverse the order so the most recent message is last
    messages = list(messages)
    messages.reverse()
    retrieval_result = agent_client.retrieve(
        retrieval_request=KnowledgeAgentRetrievalRequest(
            messages=[KnowledgeAgentMessage(role=msg["role"], content=[KnowledgeAgentMessageTextContent(text=msg.content[0].text)]) for msg in messages if msg["role"] != "system"],
            target_index_params=[KnowledgeAgentIndexParams(index_name=index_name, reranker_threshold=2.5)]
        )
    )

    # Associate the retrieval results with the last message in the conversation
    last_message = messages[-1]
    retrieval_results[last_message.id] = retrieval_result

    # Return the grounding response to the agent
    return retrieval_result.response[0].content[0].text


functions = FunctionTool({ agentic_retrieval })
toolset = ToolSet()
toolset.add(functions)
project_client.agents.enable_auto_function_calls(toolset)

In [28]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
        How do city lights reveal information about transportation infrastructure and political boundaries?
        What role does nighttime imagery play in climate research and monitoring?
    """
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value

print("Agent response:", output.replace(".", "\n"))

Agent response: City lights viewed from nighttime satellite imagery reveal information about transportation infrastructure and political boundaries in several ways:

1
 Transportation Infrastructure:
   - Road networks and highways can often be traced by the continuous lines of lights connecting urban areas

   - Airports and major transportation hubs are identifiable by clusters of lights

   - The pattern and intensity of urban lighting reflect the density and expansion of transportation systems

   These visible light patterns help map out how cities and regions are interconnected and the flow of human mobility


2
 Political Boundaries:
   - Sharp contrasts in lighting intensity along borders can indicate political boundaries, especially where economic development or infrastructure differs markedly

   - Differences in urbanization and land use on either side of a border are visible through lighting patterns

   - Nighttime imagery can highlight controlled border areas versus more 

In [29]:
import json

retrieval_result = retrieval_results.get(message.id)
if retrieval_result is None:
    raise RuntimeError(f"No retrieval results found for message {message.id}")

print("Retrieval activity")
print(json.dumps([activity.as_dict() for activity in retrieval_result.activity], indent=2))
print("Retrieval results")
print(json.dumps([reference.as_dict() for reference in retrieval_result.references], indent=2))

Retrieval activity
[
  {
    "id": 0,
    "type": "ModelQueryPlanning",
    "input_tokens": 1359,
    "output_tokens": 357
  },
  {
    "id": 1,
    "type": "AzureSearchQuery",
    "target_index": "txt_files_index",
    "query": {
      "search": "How do city lights reveal information about transportation infrastructure and political boundaries?"
    },
    "query_time": "2025-08-03T13:41:27.468Z",
    "count": 0,
    "elapsed_ms": 220
  },
  {
    "id": 2,
    "type": "AzureSearchQuery",
    "target_index": "txt_files_index",
    "query": {
      "search": "What role does nighttime imagery play in climate research and monitoring?"
    },
    "query_time": "2025-08-03T13:41:27.570Z",
    "count": 0,
    "elapsed_ms": 102
  },
  {
    "id": 3,
    "type": "AzureSearchSemanticRanker",
    "input_tokens": 0
  }
]
Retrieval results
[]


In [30]:
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What animals become active at night and how do they use darkness for survival?"
)

run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id,
    tool_choice=AgentsNamedToolChoice(type=AgentsNamedToolChoiceType.FUNCTION, function=FunctionName(name="agentic_retrieval")),
    toolset=toolset)
if run.status == "failed":
    raise RuntimeError(f"Run failed: {run.last_error}")
output = project_client.agents.messages.get_last_message_text_by_role(thread_id=thread.id, role="assistant").text.value


print("Agent response:", output.replace(".", "\n"))

Agent response: Animals that become active at night are known as nocturnal animals
 These include a wide range of species such as owls, bats, many species of rodents, some reptiles, amphibians, and various insects
 They use darkness for survival in several ways:

1
 Predation: Many nocturnal predators have adapted to hunting in low light conditions using enhanced senses like acute hearing, night vision, and echolocation to locate prey


2
 Avoiding Predators: Being active at night helps prey animals avoid diurnal predators that rely on daylight for hunting


3
 Temperature Regulation: Nighttime activity allows animals to avoid the heat of the day, which is beneficial in hot climates


4
 Reduced Competition: Nocturnal habits reduce competition for food resources by temporal separation from diurnal species


5
 Navigation and Communication: Some animals use bioluminescence or enhanced sensory signals that work better in darkness for communication and navigation


Overall, darkness confe